### GLDAS Temperature Download

Goals:

1. For each SWC station, pull GLDAS 2.0 results for the given coordinate location between 1990 and 1999.
2. For each SWC station, pull GLDAS 2.1 results for the given coordinate location between 2000 and 2020.
3. Aggregate results together into CSV file excluding no results (-9999) stations
4. For SWC stations with no results (-9999), check nearest neighboring grids for results.
5. If found, pull GLDAS 2.0 results for neighboring grid between 1990 and 1999.
6. If found, pull GLDAS 2.1 results for neighboring grid between 2000 and 2020.
7. Aggregate results together into near CSV file.

General Statistics:

* Total Stations: 5144
* Stations with GLDAS grid information: 4765
* Stations with neighboring GLDAS grid information: 318
* Stations with no GLDAS information: 61

Notes:

* Total time to download and process GLDAS is about 5 or 6 days of 24-7 single thread downloading.

* GLDAS uses a "backwards three-hour average" for temperature making querying by date a tad tricky.  For example, the final three hours of the UTC-defined day are stored under the timestamp for midnight the following day.   Some care and attention is needed to make sure the values are assigned to the proper day max and min.

* The GLDAS servers will occasionally throw one of several kinds of errors.  The most common is the "bad rods" response.  When the server returns this error things are usually in a precarious state and its best to give it a rest and return to the downloads later.  The other error is an HTTP result proclaiming an ERROR has occurred.  And the final problem is the most insidious, when results are summarily truncated with no warning.  For example, a GLDAS 2.0 call might just stop at 1997.  The notebook logic verifies that each results returns the final day of the set.

* The logic to search for neighboring grids with data creates a global 0.25 degree grid coverage and determines the nearest of the eight neighboring grids to the station location.  Then the grids are checked in order for results taking the closest grid with data.  All eight neighbor grids are checked.  Note the burden to query so many neighbor grids for the 318 locations means the grid9999 processing step takes about 6 hours to complete.

* As we assume a difference in quality between location under a grid and locations nearby to a grid, we store the results as two separate categories, **gldas** and **gldasnear**.  The difference may not matter to the final SWC product.


In [ ]:
import arcpy;
import os,sys;
import requests;
import datetime;
from time import sleep;

start_date  = datetime.datetime(1990,1,1)   + datetime.timedelta(hours=3);
end_date    = datetime.datetime(2020,12,31) + datetime.timedelta(hours=24);
target_csv  = 'gldas_20210822.csv';
target_near = 'gldasnear_20210822.csv';

results_fgdb = os.getcwd() + os.sep + '..'+ os.sep + 'results.gdb';
target_dir   = os.getcwd() + os.sep + 'gldas';

if not os.path.exists(target_dir):
    os.mkdir(target_dir);

stations = results_fgdb  + os.sep + 'SWC_Station_Universe';
stations_cnt = arcpy.GetCount_management(stations)[0];
print("  Initial SWC Station Universe Count: " + str(stations_cnt));


In [ ]:
%%time

raw_files_dir = target_dir + os.sep + 'raw';

if not os.path.exists(raw_files_dir):
    os.mkdir(raw_files_dir);

with arcpy.da.SearchCursor(
     in_table     = results_fgdb + os.sep + 'SWC_Station_Universe'
    ,field_names  = ['StationId','SHAPE@']
    ,sql_clause=(None, "ORDER BY StationID ASC")
) as incur:

    for row in incur:
        
        point = row[1].firstPoint;
        lat  = round(point.Y,8);
        lon  = round(point.X,8);
        name = row[0];
        
        target_20 = raw_files_dir + os.sep + name + '_gldas20.txt';
        target_21 = raw_files_dir + os.sep + name + '_gldas21.txt';
        
        if not os.path.exists(target_20):
                
            url = "https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/"   \
                + "access/timeseries.cgi"                              \
                + "?variable=GLDAS2:GLDAS_NOAH025_3H_v2.0:Tair_f_inst" \
                + "&startDate=" + start_date.strftime("%Y-%m-%dT%H")   \
                + "&endDate=2000-01-01T00"                             \
                + "&location=GEOM:POINT(" + str(lon) + ",%20" + str(lat) + ")" \
                + "&type=asc2";  
        
            boo_good_2000 = False;
            boo_good = True;
            with open(target_20,'wb') as f:

                with requests.get(
                     url
                    ,stream = True
                ) as r:
                    for line in r.iter_lines():
                        if line[0:6] == b'ERROR:':
                            boo_good = False;
                            break;
                        
                        if line[0:19] == b'2000-01-01T00:00:00':
                            boo_good_2000 = True;
                            
                        f.write(line+'\n'.encode());
                        
            if not boo_good:
                print("bad rods 20 for " + name + ", skipping for now");
                sleep(2);
                os.remove(target_20);
                
            elif not boo_good_2000:
                print("partial data 20 received for " + name + ", skipping for now");
                sleep(2);
                os.remove(target_20);
          
        if not os.path.exists(target_21):
            
            url = "https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/"   \
                + "access/timeseries.cgi"                              \
                + "?variable=GLDAS2:GLDAS_NOAH025_3H_v2.1:Tair_f_inst" \
                + "&startDate=1999-12-31T00"                           \
                + "&endDate=" + end_date.strftime("%Y-%m-%dT%H")       \
                + "&location=GEOM:POINT(" + str(lon) + ",%20" + str(lat) + ")" \
                + "&type=asc2"; 
            
            boo_good_2020 = False;
            boo_good = True;
            with open(target_21,'wb') as f:

                with requests.get(
                     url
                    ,stream = True
                ) as r:
                    for line in r.iter_lines():
                        if line[0:6] == b'ERROR:':
                            boo_good = False;
                            break;
                            
                        if line[0:19] == b'2020-12-31T21:00:00':
                            boo_good_2020 = True;
                            
                        f.write(line+'\n'.encode());
                        
            if not boo_good:
                print("bad rods 21 for " + name + ", skipping for now");
                sleep(2);
                os.remove(target_21);
                
            elif not boo_good_2020:
                print("partial data 21 received for " + name + ", skipping for now");
                sleep(2);
                os.remove(target_21);
            

In [ ]:
target_file = target_dir + os.sep + target_csv;

if os.path.exists(target_file):
    os.remove(target_file);

with arcpy.da.SearchCursor(
     in_table     = results_fgdb + os.sep + 'SWC_Station_Universe'
    ,field_names  = ['StationId']
) as incur:
    
    with open(target_file,'w') as outcur:
    
        for row in incur:
            name = row[0];
            
            source_file20 = raw_files_dir + os.sep + name + '_gldas20.txt';
            with open(source_file20,'r') as f20:

                writeit20 = False;
                for line in f20:
                    
                    if writeit20 and len(line) > 0:
                        (dtin20,tpin20) = line.split('\t');
                        dt20  = datetime.datetime.strptime(dtin20,"%Y-%m-%dT%H:%M:%S");
                        tpk20 = float(tpin20) ;
                        
                        if tpk20 > 0:
                            tpf20 = (tpk20 - 273.15) * 1.8000 + 32.00;

                            outcur.write('"' + name + '","' + str(dt20) + '",' + str(round(tpf20,8)) + '\n');
                            
                    if line == 'Date&Time               Data\n':
                        writeit20 = True;
                        
            source_file21 = raw_files_dir + os.sep + name + '_gldas21.txt';
            with open(source_file21,'r') as f21:

                writeit21 = False;
                for line in f21:

                    if writeit21 and len(line) > 0:
                        (dtin21,tpin21) = line.split('\t');
                        dt21  = datetime.datetime.strptime(dtin21,"%Y-%m-%dT%H:%M:%S");
                        tpk21 = float(tpin21) ;
                        
                        if tpk21 > 0:
                            tpf21 = (tpk21 - 273.15) * 1.8000 + 32.00;

                            outcur.write('"' + name + '","' + str(dt21) + '",' + str(round(tpf21,8)) + '\n');
                            
                    if line == 'Date&Time               Data\n':
                        writeit21 = True;
                        

In [ ]:
station20_9999s = [];
station21_9999s = [];

raw_files_dir = target_dir + os.sep + 'raw';
gldas9999_fc = results_fgdb + os.sep + 'gldas_9999';

if arcpy.Exists(gldas9999_fc):
    arcpy.Delete_management(gldas9999_fc);
    
arcpy.CreateFeatureclass_management(
     out_path      = os.path.dirname(gldas9999_fc)
    ,out_name      = os.path.basename(gldas9999_fc)
    ,geometry_type = "POINT"
    ,has_m         = "DISABLED"
    ,has_z         = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4269) 
);

arcpy.management.AddFields(
     in_table          = gldas9999_fc
    ,field_description = [
         ['stationId'       ,'TEXT'  ,'Station ID'       ,255]
    ]
);

arcpy.management.AddIndex(
     in_table   = gldas9999_fc
    ,fields     = 'StationId'
    ,index_name = 'StationId_IDX'
);

with arcpy.da.SearchCursor(
     in_table     = results_fgdb + os.sep + 'SWC_Station_Universe'
    ,field_names  = ['StationId']
) as incur:
    
    for row in incur:
        name = row[0];

        source_file20 = raw_files_dir + os.sep + name + '_gldas20.txt';
        with open(source_file20,'r') as f20:

            writeit20 = False;
            for line in f20:

                if writeit20 and len(line) > 0:
                    (dtin20,tpin20) = line.split('\t');
                    tpk20 = float(tpin20) ;

                    if tpk20 == -9999:
                        station20_9999s.append(name);
                    
                    break;

                if line == 'Date&Time               Data\n':
                    writeit20 = True;
                    
        source_file21 = raw_files_dir + os.sep + name + '_gldas21.txt';
        with open(source_file21,'r') as f21:

            writeit21 = False;
            for line in f21:

                if writeit21 and len(line) > 0:
                    (dtin21,tpin21) = line.split('\t');
                    tpk21 = float(tpin21) ;

                    if tpk21 == -9999:
                        station21_9999s.append(name);
                        
                    break;

                if line == 'Date&Time               Data\n':
                    writeit21 = True;
                        
if len(station20_9999s) != len(station21_9999s):
    raise Exception('9999 counts do not match between 2.0 and 2.1');
    
with arcpy.da.SearchCursor(
     in_table     = results_fgdb + os.sep + 'SWC_Station_Universe'
    ,field_names  = ['StationId','SHAPE@']
    ,sql_clause=(None, "ORDER BY StationID ASC")
) as incur:
    
    with arcpy.da.InsertCursor(
         in_table     = gldas9999_fc
        ,field_names  = ['StationId','SHAPE@']
    ) as outcur:

        for row in incur:
            name = row[0];

            if name in station20_9999s:

                outcur.insertRow((
                     row[0]
                    ,row[1]
                ));


In [ ]:
gldas_grid_fc = results_fgdb + os.sep + 'gldas_grid';

if arcpy.Exists(gldas_grid_fc):
    arcpy.Delete_management(gldas_grid_fc);

arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4269);
arcpy.cartography.GridIndexFeatures(
     out_feature_class = gldas_grid_fc
    ,polygon_width     = '0.25 degree'
    ,polygon_height    = '0.25 degree' 
    ,origin_coord      = '-180, -90'
    ,number_rows       = 720
    ,number_columns    = 1440
);

arcpy.management.AddFields(
     in_table          = gldas_grid_fc
    ,field_description = [
         ['centroidX'       ,'DOUBLE'  ,'Centroid X'     ]
        ,['centroidY'       ,'DOUBLE'  ,'Centroid Y'     ]
    ]
);

with arcpy.da.UpdateCursor(
     in_table     = gldas_grid_fc
    ,field_names  = ['SHAPE@TRUECENTROID','centroidX','centroidY']
) as cursor:
    
    for row in cursor:
        
        (ptx,pty) = row[0];
        
        row[1] = ptx;
        row[2] = pty;
        
        cursor.updateRow(row);


In [ ]:
gldas_near_tbl = results_fgdb + os.sep + 'gldas9999_near';

if arcpy.Exists(gldas_near_tbl):
    arcpy.Delete_management(gldas_near_tbl);

arcpy.analysis.GenerateNearTable(
     in_features   = gldas9999_fc
    ,near_features = gldas_grid_fc
    ,out_table     = gldas_near_tbl
    ,search_radius = '0.25 degree'
    ,closest       = 'ALL'
    ,closest_count = 9
    ,method        = 'GEODESIC'
);

arcpy.management.JoinField(
     in_data    = gldas_near_tbl
    ,in_field   = 'NEAR_FID'
    ,join_table = gldas_grid_fc
    ,join_field = 'OID'
    ,fields     = ['centroidX','centroidY']
);


In [ ]:
%%time

raw_files_dir = target_dir + os.sep + 'raw';
gldas9999_fc = results_fgdb + os.sep + 'gldas_9999';
gldas_near_tbl = results_fgdb + os.sep + 'gldas9999_near';
raw9999_files_dir = target_dir + os.sep + 'raw9999';

if not os.path.exists(raw9999_files_dir):
    os.mkdir(raw9999_files_dir);

with arcpy.da.SearchCursor(
     in_table     = gldas9999_fc
    ,field_names  = ['ObjectID','StationId']
    ,sql_clause=(None, "ORDER BY StationID ASC")
) as incur:
    
    for outer_row in incur:
        src_fid = outer_row[0];
        name    = outer_row[1];
        target_20 = raw9999_files_dir + os.sep + name + '_gldas20.txt';
        target_21 = raw9999_files_dir + os.sep + name + '_gldas21.txt';
        
        if not os.path.exists(target_20) or not os.path.exists(target_21):
            
            with arcpy.da.SearchCursor(
                 in_table     = gldas_near_tbl
                ,field_names  = ['IN_FID','NEAR_FID','NEAR_DIST','NEAR_RANK','centroidX','centroidY']
                ,sql_clause=(None, "ORDER BY NEAR_RANK ASC")
                ,where_clause = 'IN_FID = ' + str(src_fid) + ' AND NEAR_RANK > 1'
            ) as nearcur:

                boo_results = False;
                for inner_row in nearcur:

                    gridX = inner_row[4];
                    gridY = inner_row[5];
                    rank  = inner_row[3];
                    dist  = inner_row[2];
                    dist_msg = 'grid_distance=' + str(dist) + '\n';
                    
                    url = "https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/"   \
                        + "access/timeseries.cgi"                              \
                        + "?variable=GLDAS2:GLDAS_NOAH025_3H_v2.0:Tair_f_inst" \
                        + "&startDate=" + start_date.strftime("%Y-%m-%dT%H")   \
                        + "&endDate=2000-01-01T00"                             \
                        + "&location=GEOM:POINT(" + str(gridX) + ",%20" + str(gridY) + ")" \
                        + "&type=asc2";  

                    with requests.get(
                         url
                        ,stream = True
                    ) as r:
                        for line in r.iter_lines():
                            
                            if line[0:6] == b'ERROR:':
                                raise Exception('bad results for ' + name);
                                
                            str_line = str(line,'utf-8').strip();

                            val = None;
                            if str_line.find('\t') > 0:
                                (dt,val) = str_line.split('\t');
                                                               
                            if val is not None and float(val) > 0:
                                boo_results = True;
                                break;

                        if boo_results:
                            break;

            if boo_results:

                url = "https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/"   \
                    + "access/timeseries.cgi"                              \
                    + "?variable=GLDAS2:GLDAS_NOAH025_3H_v2.0:Tair_f_inst" \
                    + "&startDate=" + start_date.strftime("%Y-%m-%dT%H")   \
                    + "&endDate=2000-01-01T00"                             \
                    + "&location=GEOM:POINT(" + str(gridX) + ",%20" + str(gridY) + ")" \
                    + "&type=asc2";

                boo_good_2000 = False;
                with open(target_20,'wb') as f:
                    f.write(dist_msg.encode('utf-8'));
                    
                    with requests.get(
                         url
                        ,stream = True
                    ) as r:
                        for line in r.iter_lines():

                            if line[0:6] == b'ERROR:':
                                raise Exception('bad 20 results for ' + name);

                            if line[0:19] == b'2000-01-01T00:00:00':
                                boo_good_2000 = True;

                            f.write(line+'\n'.encode());

                if not boo_good_2000:
                    raise Exception('bad 20 results for ' + name);

                url = "https://hydro1.gesdisc.eosdis.nasa.gov/daac-bin/"   \
                    + "access/timeseries.cgi"                              \
                    + "?variable=GLDAS2:GLDAS_NOAH025_3H_v2.1:Tair_f_inst" \
                    + "&startDate=1999-12-31T00"                           \
                    + "&endDate=" + end_date.strftime("%Y-%m-%dT%H")       \
                    + "&location=GEOM:POINT(" + str(gridX) + ",%20" + str(gridY) + ")" \
                    + "&type=asc2"; 

                boo_good_2020 = False;
                with open(target_21,'wb') as f:
                    f.write(dist_msg.encode('utf-8'));

                    with requests.get(
                         url
                        ,stream = True
                    ) as r:
                        for line in r.iter_lines():

                            if line[0:6] == b'ERROR:':
                                raise Exception('bad 21 results for ' + name);

                            if line[0:19] == b'2020-12-31T21:00:00':
                                boo_good_2020 = True;

                            f.write(line+'\n'.encode());

                if not boo_good_2020:
                    raise Exception('bad results for ' + name);
                    

In [ ]:
target_file = target_dir + os.sep + target_near;
raw9999_files_dir = target_dir + os.sep + 'raw9999';

if os.path.exists(target_file):
    os.remove(target_file);

with arcpy.da.SearchCursor(
     in_table     = gldas9999_fc
    ,field_names  = ['StationId']
) as incur:
    
    with open(target_file,'w') as outcur:
    
        for row in incur:
            name = row[0];
            
            source_file20 = raw9999_files_dir + os.sep + name + '_gldas20.txt';
            source_file21 = raw9999_files_dir + os.sep + name + '_gldas21.txt';
            
            if os.path.exists(source_file20):
                
                with open(source_file20,'r') as f20:

                    grid_distance = None;
                    lat = None;
                    lon = None;
                    writeit20 = False;
                    for line in f20:

                        if line[0:14] == 'grid_distance=':
                            grid_distance = float(line[14:]);

                        elif line[0:4] == 'lat=':
                            lat = float(line[4:]);

                        elif line[0:4] == 'lon=':
                            lon = float(line[4:]);

                        elif writeit20 and len(line) > 0:
                            (dtin20,tpin20) = line.split('\t');
                            dt20  = datetime.datetime.strptime(dtin20,"%Y-%m-%dT%H:%M:%S");
                            tpk20 = float(tpin20) ;

                            if tpk20 > 0:
                                tpf20 = (tpk20 - 273.15) * 1.8000 + 32.00;

                                outcur.write('"' + name + '","' + str(dt20) + '",' + str(round(tpf20,8)) \
                                    + ',' + str(round(grid_distance,2)) + ',' + str(lon) + ',' + str(lat) + '\n');

                        if line == 'Date&Time               Data\n':
                            writeit20 = True;

                with open(source_file21,'r') as f21:

                    grid_distance = None;
                    lat = None;
                    lon = None;
                    writeit21 = False;
                    for line in f21:

                        if line[0:14] == 'grid_distance=':
                            grid_distance = float(line[14:]);

                        elif line[0:4] == 'lat=':
                            lat = float(line[4:]);

                        elif line[0:4] == 'lon=':
                            lon = float(line[4:]);

                        elif writeit21 and len(line) > 0:
                            (dtin21,tpin21) = line.split('\t');
                            dt21  = datetime.datetime.strptime(dtin21,"%Y-%m-%dT%H:%M:%S");
                            tpk21 = float(tpin21) ;

                            if tpk21 > 0:
                                tpf21 = (tpk21 - 273.15) * 1.8000 + 32.00;

                                outcur.write('"' + name + '","' + str(dt21) + '",' + str(round(tpf21,8)) \
                                    + ',' + str(round(grid_distance,2)) + ',' + str(lon) + ',' + str(lat) + '\n');

                        if line == 'Date&Time               Data\n':
                            writeit21 = True;
                            